# 🧠 GenAI Supply Chain Assistant – Executive Summary

This notebook presents a **Generative AI-powered assistant** for supply chain teams to improve decision-making around supplier delays, quality issues, and restocking needs.

The assistant combines:
- ✅ Document Understanding (simulated delivery records)
- ✅ Embedding Search with FAISS (Retrieval-Augmented Generation)
- ✅ LLM-Based Reasoning (Flan-T5) with business logic
- ✅ Hybrid Agent Logic (rules + language model)

Using these components, the assistant:
1. Parses product and supplier data
2. Flags risks like long lead times, low stock, or high defect rates
3. Suggests actions such as “Reorder”, “Flag for Delay”, or “Monitor”
4. Summarizes decisions in a structured table and visualization

This solution can scale to live enterprise systems and integrate with dashboards or ERP software to support real-time supply chain optimization.


In [1]:
#📦 These are the tools we'll use:

#pandas: to read and analyze the CSV file
#openai: to access GPT for smart suggestions
#PyMuPDF: to extract text from PDF
#sentence-transformers: to convert text into embeddings (vectors)
#faiss: to store and search those vectors

## ✅ 1. Load Environment Variables & Setup LLM

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch

In [4]:
# Load environment (.env should have OPENAI_API_KEY)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#Define LLM
llm = ChatOpenAI(model="gpt-4", openai_api_key=OPENAI_API_KEY)
output_parser = StrOutputParser()

## ✅ Step 2. Load the Dataset

In [5]:
# ✅ Step 1: Load the Dataset
# We're using the "Supply Chain Dataset" from Kaggle (by Amir Motefaker)
# This dataset contains order dates, shipment status, lead times, etc.

import pandas as pd

# Load the CSV file (replace the path with your uploaded Kaggle dataset)
df = pd.read_csv('/Users/ghazalnazari/Documents/جزوات دانشگاه آلمان/Google course/project/supply_chain_data.csv')
df.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [6]:
print(f"Shape of the dataset: {df.shape}")


Shape of the dataset: (100, 24)


In [7]:
print(df.columns.tolist())


['Product type', 'SKU', 'Price', 'Availability', 'Number of products sold', 'Revenue generated', 'Customer demographics', 'Stock levels', 'Lead times', 'Order quantities', 'Shipping times', 'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location', 'Lead time', 'Production volumes', 'Manufacturing lead time', 'Manufacturing costs', 'Inspection results', 'Defect rates', 'Transportation modes', 'Routes', 'Costs']


## ✅ Step 3: Automatically Flag Supply Chain Issues


🔍 We'll now look at each row and add warning flags if:


⏱️ Lead time is long (> 20 days)

📦 Stock is low (< 10 units)

🔥 Demand is high (> 1000 units — you can adjust)


In [11]:
def flag_issues(row):
    flags = []
    if row['Lead time'] > 20:
        flags.append("🛑 Delay")
    if row['Number of products sold'] > 1000:
        flags.append("📈 High Demand")
    if row['Stock levels'] < 10:
        flags.append("📦 Low Stock")
    if row['Defect rates'] > 3.5:
        flags.append("❌ Quality Risk")
    return flags if flags else ["✅ OK"]


# Apply flags
df['Flags'] = df.apply(flag_issues, axis=1)
flagged_df = df[df['Flags'].apply(lambda x: x != ["✅ OK"])]



In [13]:
# Show core results
df[['SKU', 'Product type', 'Stock levels', 'Lead time', 'Number of products sold', 'Defect rates', 'Flags']].head()


,SKU,Product type,Stock levels,Lead time,Number of products sold,Defect rates,Flags
0,SKU0,haircare,58,29,802,0.226410,[🛑 Delay]
1,SKU1,skincare,53,23,736,4.854068,"[🛑 Delay, ❌ Quality Risk]"
2,SKU2,haircare,1,12,8,4.580593,"[📦 Low Stock, ❌ Quality Risk]"
3,SKU3,skincare,23,24,83,4.746649,"[🛑 Delay, ❌ Quality Risk]"
4,SKU4,skincare,5,5,871,3.145580,[📦 Low Stock]


## ✅ Step 3: Load Supply Chain Knowledge (PDF) and Create Retriever


📘 We use a Supply Chain PDF as our knowledge base for GPT to give smart advice. This extracts text from the PDF.

In [14]:
import warnings
warnings.filterwarnings("ignore")


In [15]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.vectorstores import FAISS

pdf_path = "World-Bank-Supply-Chain-Management-Guidance.pdf"
loader = PyMuPDFLoader(pdf_path)
pages = loader.load()

embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(pages, embedding)
retriever = vectorstore.as_retriever()

In [16]:
print(f"Total pages loaded and embedded: {len(pages)}")


Total pages loaded and embedded: 116


In [17]:
print(f"Number of vectors in FAISS index: {vectorstore.index.ntotal}")

Number of vectors in FAISS index: 116


## ✅ Step 4: Create a Prompt Template for the Agent

In [18]:
template = """
You are an AI supply chain assistant.

Context (from policy docs):
{context}

Product Issue:
{question}

What is your action recommendation?
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm | output_parser


## ✅ Step 5: Loop Through Flagged Issues and Ask LLM

In [20]:
print(flagged_df.columns.tolist())


['Product type', 'SKU', 'Price', 'Availability', 'Number of products sold', 'Revenue generated', 'Customer demographics', 'Stock levels', 'Lead times', 'Order quantities', 'Shipping times', 'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location', 'Lead time', 'Production volumes', 'Manufacturing lead time', 'Manufacturing costs', 'Inspection results', 'Defect rates', 'Transportation modes', 'Routes', 'Costs', 'Flags']


In [21]:
for i, row in flagged_df.iterrows():
    description = (
        f"Product {row['Product type']} (SKU {row['SKU']}) from supplier {row['Supplier name']} "
        f"has a lead time of {row['Lead time']} days, current stock is {row['Stock levels']} units, "
        f"defect rate is {row['Defect rates']}%, and demand is {row.get('Demand', 'N/A')}."
    )

    print(f"\n📦 Issue Detected for SKU {row['SKU']}:")
    print(description)

    # LLM Suggestion using your retriever-based chain
    suggestion = chain.invoke({
        "context": retriever,
        "question": f"What action should we take for this supply chain issue: {description}"
    })

    print("💡 LLM Suggestion:", suggestion)



📦 Issue Detected for SKU SKU0:
Product haircare (SKU SKU0) from supplier Supplier 3 has a lead time of 29 days, current stock is 58 units, defect rate is 0.2264103608499251%, and demand is N/A.
💡 LLM Suggestion: Based on the information provided, here is an action recommendation:

1. Lead Time Analysis: With a lead time of 29 days from Supplier 3, we need to ensure that we have enough stock to cover this time period. As the demand is not provided, a sharp increase in demand could lead to stock-outs.

2. Stock Level Assessment: The current stock of 58 units may be adequate depending on the daily demand. However, without knowing the demand, it's hard to accurately determine if this is sufficient.

3. Defect Rate Examination: The defect rate is relatively low at 0.2264103608499251%. This should not significantly impact our stock levels, but it is still important to communicate this with the supplier to look for ways to reduce it further.

4. Demand Forecasting: It is crucial to estimate 

## ✅ Step 6: Display Results as a Table

In [28]:
#Store results in a list of dictionaries:
results = []

for i, row in flagged_df.iterrows():
    description = (
        f"Product {row['Product type']} (SKU {row['SKU']}) from supplier {row['Supplier name']} "
        f"has a lead time of {row['Lead time']} days, current stock is {row['Stock levels']} units, "
        f"demand is {demand_value}, and defect rate is {row['Defect rates']}%."
    )

    suggestion = chain.invoke({
        "context": retriever,
        "question": f"What action should we take for this supply chain issue: {description}"
    })

    results.append({
        "SKU": row['SKU'],
        "Product": row['Product type'],
        "Lead Time": row['Lead time'],
        "Stock": row['Stock levels'],
        "Defect Rate": row['Defect rates'],
        "Demand": demand_value,
        "Issue Summary": description,
        "AI Suggestion": suggestion
    })


In [ ]:
# Convert to clean DataFrame
final_df = pd.DataFrame(results)

# Display clean, wide table
from IPython.display import display
display(final_df)

In [24]:
#Convert to a DataFrame and display

# Show more characters per column
#pd.set_option("display.max_colwidth", None)

# Now display your table again
#results_df.head()


,SKU,Product,Lead Time,Stock,Defect Rate,Demand,Issue Summary,AI Suggestion
0,SKU0,haircare,29,58,0.226410,N/A,"Product haircare (SKU SKU0) from supplier Supplier 3 has a lead time of 29 days, current stock is 58 units, defect rate is 0.2264103608499251%, and demand is N/A.","Based on the available information, here are a few recommendations:\n\n1. Get Demand Data: First, it's important to get the demand data for the haircare product SKU0. Without knowing demand, it's impossible to accurately predict how long the current stock will last and when to reorder. \n\n2. Lead Time Safety Stock: Given the lead time of 29 days, you should maintain a safety stock that can cover at least this period. Once you have the demand data, calculate the average daily sales and multiply it by the lead time to get the safety stock level.\n\n3. Quality Control: With a defect rate of 0.2264103608499251%, you may need to work with Supplier 3 to improve quality control processes. Although the defect rate might seem low, it can still lead to customer dissatisfaction and returns, which incurs extra cost.\n\n4. Regular Review: Regularly review the lead time, stock levels, defect rate, and demand to ensure the inventory is managed optimally. \n\n5. Supplier Communication: Keep open lines of communication with Supplier 3. If the lead time consistently hinders your ability to maintain optimal stock levels, consider discussing ways to reduce it or explore alternative suppliers. \n\nRemember, these recommendations depend on the actual demand rate for the SKU0 product, which is currently not available."
1,SKU1,skincare,23,53,4.854068,N/A,"Product skincare (SKU SKU1) from supplier Supplier 3 has a lead time of 23 days, current stock is 53 units, defect rate is 4.854068026388706%, and demand is N/A.","Based on the information provided, the following actions are recommended:\n\n1. Verify Demand: The demand for the skincare product is currently not available. It's crucial to determine the demand forecast to make an informed decision. Please reach out to the sales or marketing team to get these numbers.\n\n2. Assess Lead Time: The lead time from Supplier 3 is relatively long (23 days). If the demand is high and cannot be met within this lead time, consider finding an alternate supplier with a shorter lead time or negotiate a shorter lead time with the current supplier.\n\n3. Monitor Stock Levels: With a current stock of 53 units, it's important to monitor sales closely to avoid stockouts. If sales are quick and the lead time is long, you may need to increase your safety stock.\n\n4. Address Quality Issues: The defect rate of roughly 4.85% is a concern. This could potentially lead to customer dissatisfaction and returns, which would impact supply chain efficiency and costs. It's recommended to discuss this issue with the supplier and check if they can improve their quality control.\n\n5. Review Order Frequency: Depending on the demand and the lead time, you may need to adjust your order frequency to ensure that there's always sufficient stock available.\n\nRemember, the goal is to ensure that there is enough stock to meet demand without overstocking (which can lead to increased holding costs) or understocking (which can lead to missed sales opportunities). Once the demand data is available, it's advisable to use an inventory management model like EOQ (Economic Order Quantity) to optimize order quantities."
2,SKU2,haircare,12,1,4.580593,N/A,"Product haircare (SKU SKU2) from supplier Supplier 1 has a lead time of 12 days, current stock is 1 units, defect rate is 4.580592619199229%, and demand is N/A.","Based on the information provided, the following actions are recommended:\n\n1. Contact Supplier 1: Reach out to Supplier 1 to see if there is any possibility to reduce the lead time. A lead time of 12 days could potentially cause issues if the demand for the product increases.\n\n2. Safety Stock Calculation: Since the demand is N/A, it's crucial to calculate the safety 

In [26]:
from IPython.display import display

pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping
display(results_df)



,SKU,Product,Lead Time,Stock,Defect Rate,Demand,Issue Summary,AI Suggestion
0,SKU0,haircare,29,58,0.226410,N/A,"Product haircare (SKU SKU0) from supplier Supplier 3 has a lead time of 29 days, current stock is 58 units, defect rate is 0.2264103608499251%, and demand is N/A.","Based on the available information, here are a few recommendations:\n\n1. Get Demand Data: First, it's important to get the demand data for the haircare product SKU0. Without knowing demand, it's impossible to accurately predict how long the current stock will last and when to reorder. \n\n2. Lead Time Safety Stock: Given the lead time of 29 days, you should maintain a safety stock that can cover at least this period. Once you have the demand data, calculate the average daily sales and multiply it by the lead time to get the safety stock level.\n\n3. Quality Control: With a defect rate of 0.2264103608499251%, you may need to work with Supplier 3 to improve quality control processes. Although the defect rate might seem low, it can still lead to customer dissatisfaction and returns, which incurs extra cost.\n\n4. Regular Review: Regularly review the lead time, stock levels, defect rate, and demand to ensure the inventory is managed optimally. \n\n5. Supplier Communication: Keep open lines of communication with Supplier 3. If the lead time consistently hinders your ability to maintain optimal stock levels, consider discussing ways to reduce it or explore alternative suppliers. \n\nRemember, these recommendations depend on the actual demand rate for the SKU0 product, which is currently not available."
1,SKU1,skincare,23,53,4.854068,N/A,"Product skincare (SKU SKU1) from supplier Supplier 3 has a lead time of 23 days, current stock is 53 units, defect rate is 4.854068026388706%, and demand is N/A.","Based on the information provided, the following actions are recommended:\n\n1. Verify Demand: The demand for the skincare product is currently not available. It's crucial to determine the demand forecast to make an informed decision. Please reach out to the sales or marketing team to get these numbers.\n\n2. Assess Lead Time: The lead time from Supplier 3 is relatively long (23 days). If the demand is high and cannot be met within this lead time, consider finding an alternate supplier with a shorter lead time or negotiate a shorter lead time with the current supplier.\n\n3. Monitor Stock Levels: With a current stock of 53 units, it's important to monitor sales closely to avoid stockouts. If sales are quick and the lead time is long, you may need to increase your safety stock.\n\n4. Address Quality Issues: The defect rate of roughly 4.85% is a concern. This could potentially lead to customer dissatisfaction and returns, which would impact supply chain efficiency and costs. It's recommended to discuss this issue with the supplier and check if they can improve their quality control.\n\n5. Review Order Frequency: Depending on the demand and the lead time, you may need to adjust your order frequency to ensure that there's always sufficient stock available.\n\nRemember, the goal is to ensure that there is enough stock to meet demand without overstocking (which can lead to increased holding costs) or understocking (which can lead to missed sales opportunities). Once the demand data is available, it's advisable to use an inventory management model like EOQ (Economic Order Quantity) to optimize order quantities."
2,SKU2,haircare,12,1,4.580593,N/A,"Product haircare (SKU SKU2) from supplier Supplier 1 has a lead time of 12 days, current stock is 1 units, defect rate is 4.580592619199229%, and demand is N/A.","Based on the information provided, the following actions are recommended:\n\n1. Contact Supplier 1: Reach out to Supplier 1 to see if there is any possibility to reduce the lead time. A lead time of 12 days could potentially cause issues if the demand for the product increases.\n\n2. Safety Stock Calculation: Since the demand is N/A, it's crucial to calculate the safety 